<a href="https://colab.research.google.com/github/scaairesearch/da_demo/blob/main/gradio_da_demo_mnist_mnist_m_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.0 MB/s et

In [48]:
import gradio as gr
import os
from PIL import Image
from torchvision import datasets,transforms
import random
import torch

In [ ]:
def display_image():
    # Load the image from a local file
    image = Image.open("/content/mnist-m.JPG")
    return image

In [40]:
img_size = 28 # for mnist
cpu_batch_size = 10
# cuda_batch_size = 128
# EPOCHS = 1 # 15 if cuda else 2

# target_drive_path = '/content/gdrive/MyDrive/mnist_m/dataset/mnist_m.tar.gz'
# model_drive_path = '/content/gdrive/MyDrive/da_demo/cv/models'
# demo_data_path = '/content/gdrive/MyDrive/da_demo/data'

# data_base_path = '/content/data'
# source_dataset_name = 'MNIST'
# target_dataset_name = 'mnist_m'
# source_image_root = os.path.join(data_base_path, source_dataset_name)
# target_image_root = os.path.join(data_base_path, target_dataset_name)
# model_root = 'models'
# cuda = True
# cudnn.benchmark = True
# lr = 1e-3
# batch_size = 128
# image_size = 28
# n_epoch = 100

### SOURCE


In [79]:
# Test Phase transformations
test_transforms = transforms.Compose([
                                       transforms.Resize(img_size),
                                       transforms.ToTensor(),# converts to tesnor
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])

display_transforms = transforms.Compose([
                                       transforms.Resize(img_size),
                                      #  transforms.ToTensor(),# converts to tesnor
                                      #  transforms.Normalize((0.1307,), (0.3081,)),
                                       transforms.ToPILImage()
                                       ])

test = datasets.MNIST('./data',
                      train=False,
                      download=True,
                      transform=test_transforms)

dataloader_args = dict(shuffle=True, batch_size=cpu_batch_size)

mnist_loader = torch.utils.data.DataLoader(
    dataset = test,
    **dataloader_args
)

In [83]:
def get_device():
    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"
    print("Device Selected:", device)
    return device

device = get_device()
print(device)

Device Selected: cpu
cpu


In [ ]:
def get_mnist_images(num=10):
  images, labels = next(iter(mnist_loader))
  for image, label in zip (images,labels):


In [81]:
def classify_image(image, num_classes=3):
  # transforming image and getting prediction from model
  transformed_image = test_transforms(image)
  image_tensor = transformed_image.to(device).unsqueeze(0)#transform(torch.tensor(image).to(DEVICE)).unsqueeze(0) # making it a batch

  # sending it to model to get prediction
  logits = loaded_model(image_tensor) # logits
  output = F.softmax(logits.view(-1)) #F.softmax(output.flatten(), dim=-1) #

  confidences = [(classes[i], float(output[i])) for i in range(len(classes))]
  confidences.sort(key=lambda x: x[1], reverse=True)
  confidences = OrderedDict(confidences[:num_classes])
  label = torch.argmax(output).item()

In [80]:


with gr.Blocks() as demo:

  with gr.Tab("MNIST-M : inference NonDANN and DANN"):
    with gr.Row():
      radio_gradcam = gr.Radio(["Baseline (Non-DANN)", "DANN"],
                               label="Select the model you want to use.",
                               value="Baseline (Non-DANN)",  # Set default value
                               scale=2)

    with gr.Row():
      with gr.Column():
        input_image_classify = gr.Image(label="Classify Digit", type = "pil", height = 256, width = 256)
        button_classify = gr.Button("Submit to Classify Image", visible = True)



      with gr.Column():
        label_classify = gr.Label(num_top_classes=10, visible = True)

    with gr.Row():
      # gr.Markdown("Select the Image")
      for batch in mnist_loader:
        images, labels = batch[0],batch[1]
        image_pils = [display_transforms(img) for img in images]
        # print(labels)
        # for img,label in zip (images, labels):
        #   img_pil = display_transforms(img)
        #   # with gr.Column():
        #   gr.Examples([img_pil],inputs=[input_image_classify],label=label.item())
        # break
        gr.Examples(image_pils,inputs=[input_image_classify], label = "Select an example MNIST Image")
        break





  ##########################
  with gr.Tab("Introduction"):
    gr.Markdown("## Domain Adaptation in Deep Networks - Demonstration")
    gr.Markdown(
        '''
        Source - MNIST
        ------
        - The MNIST database (Modified National Institute of Standards and Technology database) is a large collection of handwritten digits.
        - It has a training set of 60,000 examples, and a test set of 10,000 examples.
        - 28 x 28 size
        - 1 channel

        '''
        )
    gr.Markdown(
        '''
        Target - MNIST-M
        -------
        - MNIST-M is created by combining MNIST digits with the patches randomly extracted from color photos of BSDS500 as their background.
        - It contains 59,001 training and 90,001 test images.
        - 28 x 28 size
        - 3 channels
        '''
    )
    with gr.Row():
        with gr.Column():
             image_output = gr.Image(value=display_image(), label = "source and target",height = 512, width = 512, show_label = True)
  ################

  button_classify.click(fn=classify_image, inputs =[input_image_classify,slider_classify_num_classes,checkbox_gradcam_classify,dropdown_gradcam_classify_layer,slider_gradcam_classify_opacity], outputs = [label_classify,gallery_gradcam_classify,gallery_gradcam_classify])


  # radio_gradcam.change(fn=view_gradcam_images, inputs=radio_gradcam, outputs=[slider_gradcam_num_images, dropdown_gradcam_layer,slider_gradcam_opacity,button_gradcam, output_gallery_gradcam])



demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://444ece5ec8ab96a3ab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# import gradio as gr

# def display_image(image):
#     return image

# iface = gr.Interface(
#     fn=display_image,
#     inputs=gr.inputs.Image(type="pil"),
#     outputs=gr.outputs.Image(type="pil"),
#     title="Image Display",
#     description="Upload an image to display it."
# )

# iface.launch()


In [14]:
# Launch the Gradio app
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0d6269534cc6fe1ba0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [38]:
import random
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import gradio as gr

# Dummy dataset and dataloader for demonstration
class CustomDataset(Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        # Assuming the dataset consists of images and labels
        self.data = [
            ('/content/mnist-m.JPG', 0)
        ]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image, label, img_path

# Define transforms
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
])

# Initialize dataset and dataloader
dataset = CustomDataset(transform=transform)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# Function to get 10 random images
def get_random_images():
    data_iter = iter(dataloader)
    images, labels, paths = next(data_iter)
    images = [transforms.ToPILImage()(img) for img in images]
    return images, paths

# Dummy model for demonstration
def classify_digit(image):
    # Replace this with actual model inference code
    return "Predicted Label"

# Initialize 10 random images
images, paths = get_random_images()

# Create Gradio interface
def display_images(image_path):
    # Find the corresponding image from paths
    selected_image = None
    for img, path in zip(images, paths):
        if path == image_path:
            selected_image = img
            break

    if selected_image:
        return selected_image, classify_digit(selected_image)

with gr.Blocks() as demo:
    gr.Markdown("## Select an Image and Classify Digit")

    with gr.Row():
        # Display 10 random images
        for img, path in zip(images, paths):
            with gr.Column():
                gr.Image(value=img, label=path, interactive=True).click(
                    display_images, inputs=path, outputs=["image_output", "label_classify"]
                )

    with gr.Row():
        image_output = gr.Image(label="Selected Image")
        label_classify = gr.Label(label="Classification Result")

# Launch the Gradio app
demo.launch()


AttributeError: 'Image' object has no attribute 'click'